In [1]:
import os

from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

### Create the agent
An agent consists of a large language model (LLM) and a set of tools. Tools can simply be Python functions that the agent can call to perform specific tasks.

In [2]:
load_dotenv()

# initialize model using custom endpoint
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [3]:
# define a simple function as a tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [4]:
# create the agent (LLM + tools)
agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant.",
)

### Run the agent
Running the agent involves sending it a user message and getting back a response. The response may include calls to tools, which the agent can use to get additional information before formulating its final response.

The response from the agent includes all messages exchanged during the conversation, including tool calls and their outputs.

In [5]:
from pprint import pprint

In [6]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Auckland?"}]}
)
pprint(response)

{'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='f2c0cabf-e272-41b7-8fc0-6bb0628b0db9'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgYWiUQ6PkthQPm71o5PnTH00F8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48dd-fba4-7d91-b68d-ba2c1d2258fc-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_kkIkTQv3E9KQzvCrnEWjTuaL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 7

### Agent memory
Adding memory means that we can continue the conversation over multiple interactions, with the agent remembering previous messages and tool calls.

In [7]:
from langgraph.checkpoint.memory import InMemorySaver

In [8]:
# create the agent with a checkpointer to enable memory
agent = create_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=InMemorySaver(),
    system_prompt="You are a helpful assistant.",
)

In [9]:
# create a config dictionary to specify thread ID
config = {"configurable": {"thread_id": "1"}}

# invoke the agent with memory
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Auckland?"}]},
    config=config,
)
pprint(response)

{'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='b60c89b6-6a26-4f3a-abdf-88a6037beb33'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgZB2TZNb413jrCzIkw5OEXx1y2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48de-007c-7220-b69e-3bdfaa97d023-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_7J4tVyzrsfeYCpr2Mbyqb4TQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 7

Now, invoke the agent again with a follow-up question that references the earlier part of the conversation. The agent should be able to recall the previous discussion and provide a coherent response.

In [10]:
# invoke again to see memory in action
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about in Cape Town?"}]},
    config=config,
)
pprint(response)

{'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='b60c89b6-6a26-4f3a-abdf-88a6037beb33'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgZB2TZNb413jrCzIkw5OEXx1y2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48de-007c-7220-b69e-3bdfaa97d023-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_7J4tVyzrsfeYCpr2Mbyqb4TQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 7

Notice how the conversation grows, without us having to explicitly pass in previous messages.

By using a checkpointer, the agent is able to remember previous interactions in the same thread.

### `stream_mode`
This argument controls the format of the response from the agent. The most common options are:
  - `"values"`: emit all values in the state after each step
  - `"updates"`: emit only the updated values in the state after each step
  - `"messages"`: emit LLM messages token-by-token, together with metadata for any LLM invocations inside nodes or tasks

In [11]:
# Use stream_mode="values" to show full state after each step
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What about Wellington? It must be windy there"}
        ]
    },
    config=config,
    stream_mode="values",
)

pprint(response)

{'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='b60c89b6-6a26-4f3a-abdf-88a6037beb33'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgZB2TZNb413jrCzIkw5OEXx1y2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48de-007c-7220-b69e-3bdfaa97d023-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_7J4tVyzrsfeYCpr2Mbyqb4TQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 7

In [12]:
# Use stream_mode="updates" to show only updated values in the state after each step
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "what about London? It can't be sunny too?"}
        ]
    },
    config=config,
    stream_mode="updates",
)

pprint(response)

[{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 235, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgbKuSEurhAWQkCI4gquj0Ys7MI', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48de-0b3b-7db3-8cb7-bdc6de42b59c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 'call_1PBJLQT0qdH72wp6OIjlUMgb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 235, 'output_tokens': 15, 'total_tokens': 250, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}},
 {'tools': {'messages': [ToolM

In [13]:
# helper code to extract all messages from the response into a flat list of messages
all_messages = [
    message for d in response for key in d for message in d[key]["messages"]
]

pprint(all_messages)

[AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 235, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CplgbKuSEurhAWQkCI4gquj0Ys7MI', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48de-0b3b-7db3-8cb7-bdc6de42b59c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 'call_1PBJLQT0qdH72wp6OIjlUMgb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 235, 'output_tokens': 15, 'total_tokens': 250, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 ToolMessage(content="It's always sunny in London!", name

In [14]:
# Use stream_mode="messages" to return messages token-by-token
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Does this sunny weather make you suspicious?"}
        ]
    },
    config=config,
    stream_mode="messages",
)

pprint(response)

[(AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019b48de-1023-7381-a5b4-4ee048b2e140'),
  {'checkpoint_ns': 'model:7881cde0-0862-c75d-3b35-fa19cb8f7a4f',
   'langgraph_checkpoint_ns': 'model:7881cde0-0862-c75d-3b35-fa19cb8f7a4f',
   'langgraph_node': 'model',
   'langgraph_path': ('__pregel_pull', 'model'),
   'langgraph_step': 21,
   'langgraph_triggers': ('branch:to:model',),
   'ls_model_name': 'gpt-4.1-mini',
   'ls_model_type': 'chat',
   'ls_provider': 'openai',
   'ls_temperature': None,
   'thread_id': '1'}),
 (AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019b48de-1023-7381-a5b4-4ee048b2e140'),
  {'checkpoint_ns': 'model:7881cde0-0862-c75d-3b35-fa19cb8f7a4f',
   'langgraph_checkpoint_ns': 'model:7881cde0-0862-c75d-3b35-fa19cb8f7a4f',
   'langgraph_node': 'model',
   'langgraph_path': ('__pregel_pull', 'model'),
   'langgraph_step': 21,
   'langgraph_triggers': ('branch:to:model'

In [ ]:
# Use stream_mode="messages" to return messages token-by-token
stream = agent.stream(
    {"messages": [{"role": "user", "content": "How's the weather in Auckland?"}]},
    config=config,
    stream_mode="updates",
)

for event in stream:
    pprint(event)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 737, 'total_tokens': 753, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-Cpln09uS0Qir5PGxP6txhnQtqmE7E', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b48e4-1a06-7632-bd19-56fb0da1e290-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_NokluDlRdFyszqeQP7oe2FoQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 737, 'output_tokens': 16, 'total_tokens': 753, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
{'tools': {'messages': [ToolMe

### `agent.get_state()`
This method retrieves the current state of the agent, including all messages exchanged so far, tool calls made, and their outputs. This is useful for understanding the context of the conversation and how the agent arrived at its current response.

In [14]:
state = agent.get_state(config)
pprint(state)

# what can we do with the state object?
help(state)

StateSnapshot(values={'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='ddae3481-d896-4395-a9fb-909cfdf601ae'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CpKcPTONCURxGsJ4rPSzJ4r5aKNuk', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42aa-6385-7e02-b8b2-96910b8599c6-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_dWUfaYmkS9eMzpzghRQQEwN5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tok

In [15]:
pprint(state.values)

{'messages': [HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='ddae3481-d896-4395-a9fb-909cfdf601ae'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CpKcPTONCURxGsJ4rPSzJ4r5aKNuk', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42aa-6385-7e02-b8b2-96910b8599c6-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_dWUfaYmkS9eMzpzghRQQEwN5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 7

In [16]:
messages = state.values.get("messages")
pprint(messages)

[HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='ddae3481-d896-4395-a9fb-909cfdf601ae'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CpKcPTONCURxGsJ4rPSzJ4r5aKNuk', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42aa-6385-7e02-b8b2-96910b8599c6-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_dWUfaYmkS9eMzpzghRQQEwN5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 16, 'total_tokens': 73, 'input_token_details': 

In [19]:
# example of iterating through messages to render a conversation between a user and the agent
for m in messages:
    print(f"==== {m.type} ====")
    print(f"Message: {m.content}")
    try:
        print(m.tool_calls)
    except AttributeError:
        print("No tool calls")
    pprint(m)
    print()

==== human ====
Message: what is the weather in Auckland?
No tool calls
HumanMessage(content='what is the weather in Auckland?', additional_kwargs={}, response_metadata={}, id='ddae3481-d896-4395-a9fb-909cfdf601ae')

==== ai ====
Message: 
[{'name': 'get_weather', 'args': {'city': 'Auckland'}, 'id': 'call_dWUfaYmkS9eMzpzghRQQEwN5', 'type': 'tool_call'}]
AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 57, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CpKcPTONCURxGsJ4rPSzJ4r5aKNuk', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b42aa-6385-7e02-b8b2-96910b8599c6-0', tool_calls=[{'name'